In [12]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

#Dict of accuracies
d = {}

p = pd.read_csv("phishing.csv")
#p.info()
#print(type(p))

p.columns = ['having_IPhaving_IP_Address',
'URLURL_Length',
'Shortining_Service',
'having_At_Symbol',
'double_slash_redirecting',
'Prefix_Suffix',
'having_Sub_Domain',
'SSLfinal_State',
'Domain_registeration_length',
'Favicon',
'port',
'HTTPS_token',
'Request_URL',
'URL_of_Anchor',
'Links_in_tags',
'SFH',
'Submitting_to_email',
'Abnormal_URL',
'Redirect',
'on_mouseover',
'RightClick',
'popUpWidnow',
'Iframe',
'age_of_domain',
'DNSRecord',
'web_traffic',
'Page_Rank',
'Google_Index',
'Links_pointing_to_page',
'Statistical_report',
'Result']
target = p["Result"].T

data = p.drop('Result', axis = 1)
#print(data)
#print(target)

#KNN classification

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(data, target, test_size=0.3,random_state=21, stratify=target)

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print("Test set predictions:\n {}".format(y_pred))
print("accuracy of KNN: ",knn.score(X_test, y_test))

d['accuracy of KNN'] = knn.score(X_test, y_test)

#len(X_test)

y_test = np.asarray(y_test)
X_test = np.asarray(X_test)

misclassified = np.where(y_test != y_pred)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )
print("######################################")
##Decision Trees

from sklearn.tree import DecisionTreeClassifier

# Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
dt = DecisionTreeClassifier(max_depth=25, random_state=1)

# Fit dt to the training set
dt.fit(X_train, y_train)

# Predict test set labels
y_pred_dt = dt.predict(X_test)

from sklearn.metrics import accuracy_score


# Compute test set accuracy  
acc = accuracy_score(y_test, y_pred_dt)
print("accuracy of DT: {}".format(acc))

d['accuracy of DT'] = acc

misclassified = np.where(y_test != y_pred_dt)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )
print("######################################")
## Ensemble bagging

# Import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Import BaggingClassifier
from sklearn.ensemble import BaggingClassifier

# Instantiate dt
dt_b = DecisionTreeClassifier(random_state=1, max_depth = 25)

# Instantiate bc
bc = BaggingClassifier(base_estimator=dt_b, 
                       n_estimators=50,
                       oob_score=True,
                       random_state=1)

# Fit bc to the training set 
bc.fit(X_train, y_train)

# Predict test set labels
y_pred_dt_b = bc.predict(X_test)

# Evaluate test set accuracy
acc_test = accuracy_score(y_test, y_pred_dt_b)

d['accuracy of Bagging DT'] = acc_test


# Evaluate OOB accuracy
acc_oob = bc.oob_score_

d['oob accuracy of Bagging DT'] = acc_oob

# Print acc_test and acc_oob
print('accuracy of DT Bagging: {}, OOB accuracy: {}'.format(acc_test, acc_oob))

misclassified = np.where(y_test != y_pred_dt_b)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )
print("######################################")


##Ensemble Voting

from sklearn.linear_model import LogisticRegression
SEED = 1


# Instantiate knn
knn = KNeighborsClassifier(n_neighbors=5)

# Instantiate dt
dt = DecisionTreeClassifier(random_state=SEED, max_depth = 25)


classifiers = [ ('K Nearest Neighbours', knn), ('Classification Tree', dt)]
for clf_name, clf in classifiers:  
    # Fit clf to the training set
    clf.fit(X_train, y_train)  
    # Predict y_pred
    y_pred = clf.predict(X_test)    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)  
    d['accuracy of'+ ' ' + clf_name +' in Voting'] = accuracy
    # Evaluate clf's accuracy on the test set
    print('{} : {}'.format(clf_name, accuracy))
    

# Import VotingCLassifier from sklearn.ensemble
from sklearn.ensemble import VotingClassifier

# Instantiate a VotingClassifier vc 
vc = VotingClassifier(estimators=classifiers)     

# Fit vc to the training set
vc.fit(X_train, y_train)   

# Evaluate the test set predictions
y_pred_vc = vc.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Voting Classifier: {}'.format(accuracy))

d['accuracy of Voting classifiers collectively'] = accuracy

misclassified = np.where(y_test != y_pred_vc)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )
print("######################################")

##Logistic regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C = 100, solver='newton-cg', max_iter = 25)

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred_LoR = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_LoR)
print('Logistic Regression: {}'.format(accuracy))

d['accuracy of Logistic Regression'] = accuracy

misclassified = np.where(y_test != y_pred_LoR)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )
print("######################################")

##Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 30, random_state = 13)

# Fit the classifier to the training data
RFC.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred_RFC = RFC.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_RFC)
print('Random Forest Classifier Accuracy: {}'.format(accuracy))

d['accuracy of Random Forest Classifier'] = accuracy

misclassified = np.where(y_test != y_pred_RFC)
print("misclassfied elements")
print(misclassified)
print("misclassfied elements count: ", len(misclassified[0]) )

print("BEST MODEL")

print("######################################")


print('\n')
print('\n')
for i in d:
    print(i,' : ' ,d[i])

Test set predictions:
 [ 1 -1 -1 ... -1 -1  1]
accuracy of KNN:  0.9529695507989147
misclassfied elements
(array([   4,   24,   98,  106,  130,  158,  201,  204,  206,  222,  238,
        252,  305,  308,  317,  328,  347,  352,  370,  406,  469,  503,
        509,  515,  518,  526,  536,  607,  634,  636,  649,  665,  667,
        702,  719,  722,  725,  813,  827,  871,  893,  896,  944,  947,
        988, 1021, 1023, 1041, 1045, 1046, 1057, 1058, 1081, 1110, 1132,
       1155, 1157, 1189, 1201, 1203, 1238, 1252, 1274, 1279, 1307, 1330,
       1399, 1402, 1419, 1466, 1467, 1477, 1483, 1544, 1545, 1557, 1562,
       1571, 1609, 1622, 1661, 1671, 1712, 1721, 1747, 1752, 1754, 1755,
       1778, 1787, 1810, 1840, 1854, 1883, 1940, 1941, 1948, 2037, 2082,
       2138, 2140, 2162, 2216, 2266, 2276, 2338, 2339, 2361, 2381, 2385,
       2391, 2393, 2449, 2478, 2507, 2526, 2534, 2585, 2630, 2631, 2635,
       2706, 2740, 2760, 2776, 2782, 2803, 2808, 2817, 2822, 2826, 2828,
       2856, 2857

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Voting Classifier: 0.967741935483871
misclassfied elements
(array([   6,   24,   41,   53,   99,  130,  158,  201,  204,  206,  222,
        237,  238,  252,  257,  305,  317,  328,  370,  406,  434,  469,
        509,  518,  521,  607,  661,  665,  667,  702,  719,  722,  725,
        813,  896,  944,  972,  986,  999, 1016, 1021, 1041, 1046, 1057,
       1074, 1081, 1084, 1132, 1155, 1189, 1201, 1252, 1274, 1279, 1287,
       1399, 1434, 1466, 1467, 1483, 1544, 1545, 1562, 1671, 1702, 1712,
       1715, 1721, 1747, 1752, 1754, 1810, 1840, 1842, 1854, 1883, 1941,
       1948, 1953, 1998, 2004, 2037, 2077, 2082, 2083, 2124, 2140, 2162,
       2205, 2216, 2266, 2276, 2339, 2361, 2381, 2385, 2393, 2449, 2464,
       2478, 2507, 2526, 2534, 2552, 2585, 2630, 2631, 2633, 2776, 2803,
       2808, 2822, 2826, 2828, 2856, 2857, 2866, 2894, 2926, 2937, 2967,
       3041, 3060, 3066, 3090, 3094, 3126, 3136, 3162, 3199, 3298, 3310]),)
misclassfied elements count:  132
###########################